In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../Cleaning_Engineering/added_demand_estimates.csv")

In [3]:
df['dynamic_price'] = df['base_price'] * (1 + df['estimated_eta'] * df['estimated_demand'])

In [4]:
df

,cab_type,source,destination,car_type,weekday,rush_hour,is_raining,temp_groups,surge_multiplier,price,base_price,estimated_eta,estimated_a,estimated_b,estimated_demand,dynamic_price
0,Uber,Theatre District,Fenway,Wheel Chair Accessible,1,0,0,40,1.0,9.5,7.0,2.118686e-01,0.007058,9.375647,9.380028,20.911332
1,Uber,Theatre District,South Station,Wheel Chair Accessible,1,0,0,40,1.0,7.0,6.5,2.118686e-01,0.007058,9.375647,9.380320,19.418068
2,Uber,Theatre District,Haymarket Square,Wheel Chair Accessible,1,0,0,40,1.0,8.5,7.0,2.118686e-01,0.007058,9.375647,9.380132,20.911487
3,Uber,Theatre District,Boston University,Wheel Chair Accessible,1,0,0,40,1.0,7.0,7.0,2.118686e-01,0.007058,9.375647,9.380320,20.911766
4,Uber,Theatre District,Boston University,Wheel Chair Accessible,1,0,0,40,1.0,13.5,7.0,2.118686e-01,0.007058,9.375647,9.379713,20.910866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574173,Lyft,Boston University,Theatre District,Luxury SUV,1,0,0,30,1.0,32.5,27.0,-6.537759e-08,0.999991,19.999991,20.999981,26.999963
574174,Uber,Boston University,West End,Luxury SUV,1,0,0,30,1.0,34.0,27.0,-6.537759e-08,0.999991,19.999991,20.999981,26.999963
574175,Uber,Boston University,Back Bay,Luxury SUV,1,0,0,30,1.0,27.5,23.0,-6.537759e-08,0.999991,19.999991,20.999981,22.999968
574176,Lyft,Boston University,West End,Luxury SUV,1,0,0,30,1.0,32.5,27.0,-6.537759e-08,0.999991,19.999991,20.999981,26.999963


Equations

Q(Quantity Demanded)= a ⋅P^−η + b

Price = Base Price ×(1 +η × Demand Factor)


Limitations: It is said that the problem we are facing is due to the low value of a. However, if we assume a large value from the beginning and run MCMC, it hardly works due to memory issues. So, we estimate a to a lower decimal point, calculate the A and demand, and then multiply it by a certain amount of constant (according to market conditions).



Solution: We ended up multiplying the current DEMAND column by a constant, which we’re told is a common practice for companies to keep A around 100. (We’re told that this number can be set arbitrarily by the company.) However, our current estimate is that A is filled with a number like 0.24242. So why not multiply A by a constant (around 600), make A around 100, and then recalculate DEMAND to apply your modeling?

In [5]:
df['estimated_a'].describe()


count    574178.000000
mean          0.280766
std           0.439563
min           0.000452
25%           0.001499
50%           0.002775
75%           0.911083
max           0.999991
Name: estimated_a, dtype: float64

create visual for intial data showing the lack of diversity in demand values, trying constant 600

In [6]:
adj_df = df.copy()
const = 600
adj_df['estimated_a'] = adj_df['estimated_a'] * const

In [8]:
# Assuming base_price, elasticity, and demand are columns in a DataFrame named df
adj_df['estimated_demand'] = adj_df['estimated_a'] * np.exp(-np.abs(adj_df['estimated_eta']) * np.log(adj_df['price'])) + adj_df['estimated_b']
adj_df['dynamic_price'] = adj_df['base_price'] * (1 + adj_df['estimated_eta'] * adj_df['estimated_demand'])

In [11]:
df.sample(10)

,cab_type,source,destination,car_type,weekday,rush_hour,is_raining,temp_groups,surge_multiplier,price,base_price,estimated_eta,estimated_a,estimated_b,estimated_demand,dynamic_price
108857,Uber,Financial District,Boston University,Luxury,1,0,0,40,1.0,34.0,15.0,1.537250e-07,0.999949,19.999950,20.999899,15.000048
145154,Uber,Haymarket Square,West End,Luxury,1,0,1,40,1.0,15.0,10.5,2.170230e-01,0.001434,15.989006,15.989803,46.936624
108009,Lyft,Northeastern University,Financial District,Luxury,0,0,0,30,1.0,26.0,16.0,2.334382e-07,0.999936,19.999932,20.999867,16.000078
133415,Lyft,South Station,Financial District,Luxury,1,0,1,40,1.0,10.5,10.5,2.245373e-01,0.000889,19.127042,19.127566,55.595945
51713,Lyft,Theatre District,North End,Luxury,1,0,0,30,1.0,19.5,13.5,-7.573225e-06,0.911083,19.917230,20.828292,13.497871
515967,Uber,Northeastern University,North Station,Luxury SUV,1,0,0,20,1.0,32.5,29.5,3.717222e-08,0.999990,19.999989,20.999979,29.500023
420693,Uber,Northeastern University,Financial District,Base,1,0,0,30,1.0,12.5,6.5,2.185133e-01,0.002156,10.554770,10.556012,21.493088
121806,Uber,Back Bay,North End,Luxury,1,1,1,40,1.0,23.5,16.5,2.263005e-01,0.000632,19.456694,19.457003,89.151652
256376,Lyft,Haymarket Square,Back Bay,Shared,1,0,0,40,1.0,7.0,3.0,1.997605e-01,0.001790,6.186371,6.187585,6.708105
494403,Lyft,North End,Beacon Hill,Luxury SUV,1,0,0,40,1.0,30.0,26.0,9.787006e-09,0.999985,19.999984,20.999968,26.000005


In [13]:
print('sum of original price: ', df['price'].sum())
print('sum of adjusted price: ', adj_df['dynamic_price'].sum())

sum of original price:  9500396.629999999
sum of adjusted price:  17349308.98567083


In [55]:
df[(df['source'] == 'South Station') & (df['destination'] == 'Theatre District') & (df['car_type'] == 'Base')]['estimated_demand'].describe()

count    1337.000000
mean        8.221610
std         0.000629
min         8.219048
25%         8.221185
50%         8.221657
75%         8.222206
max         8.222518
Name: estimated_demand, dtype: float64

In [56]:
adj_df[(adj_df['source'] == 'South Station') & (adj_df['destination'] == 'Theatre District') & (adj_df['car_type'] == 'Base')]['estimated_demand'].describe()

count    1337.000000
mean       19.276486
std         0.377641
min        17.739701
25%        19.021680
50%        19.304650
75%        19.634417
max        19.821641
Name: estimated_demand, dtype: float64

In [59]:
df[(df['source'] == 'South Station') & (df['destination'] == 'Theatre District') & (df['car_type'] == 'Base')]['price'].describe()


count    1337.000000
mean        8.223635
std         1.427419
min         6.500000
25%         7.000000
50%         8.000000
75%         9.000000
max        16.000000
Name: price, dtype: float64

In [60]:
df[(df['source'] == 'South Station') & (df['destination'] == 'Theatre District') & (df['car_type'] == 'Base')]['dynamic_price'].describe()


count    1337.000000
mean       18.214960
std         0.000897
min        18.211311
25%        18.214355
50%        18.215027
75%        18.215810
max        18.216255
Name: dynamic_price, dtype: float64

In [61]:
adj_df[(adj_df['source'] == 'South Station') & (adj_df['destination'] == 'Theatre District') & (adj_df['car_type'] == 'Base')]['dynamic_price'].describe()

count    1337.000000
mean       33.967038
std         0.538101
min        31.777275
25%        33.603965
50%        34.007168
75%        34.477053
max        34.743829
Name: dynamic_price, dtype: float64

In [57]:
adj_df

,cab_type,source,destination,car_type,date_time,weekday,rush_hour,is_raining,temp_groups,surge_multiplier,price,base_price,estimated_eta,estimated_a,estimated_b,estimated_demand,dynamic_price
0,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 04:43:18.387,1,0,0,40,1.0,9.5,6.5,0.224242,34.221533,8.622001,29.278334,49.175217
1,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 04:49:39.601,1,0,0,40,1.0,9.5,6.5,0.224242,34.221533,8.622001,29.278334,49.175217
2,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 06:25:02.313,1,0,0,40,1.0,8.5,6.5,0.224242,34.221533,8.622001,29.800011,49.935597
3,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 06:30:02.379,1,0,0,40,1.0,7.0,6.5,0.224242,34.221533,8.622001,30.742422,51.309228
4,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 07:16:52.978,1,1,0,40,1.0,7.0,6.5,0.224242,34.221533,8.622001,30.742422,51.309228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574173,Uber,Northeastern University,Back Bay,Luxury SUV,2018-12-17 04:30:06.790,1,0,0,30,1.0,25.0,25.0,0.000002,599.924015,19.999876,619.919271,25.037071
574174,Lyft,Northeastern University,Back Bay,Luxury SUV,2018-12-17 05:00:04.215,1,0,0,30,1.0,26.0,25.0,0.000002,599.924015,19.999876,619.919215,25.037071
574175,Uber,Northeastern University,Back Bay,Luxury SUV,2018-12-17 05:20:10.955,1,0,0,30,1.0,26.5,25.0,0.000002,599.924015,19.999876,619.919187,25.037071
574176,Uber,Northeastern University,Back Bay,Luxury SUV,2018-12-17 05:50:10.584,1,0,0,30,1.0,26.5,25.0,0.000002,599.924015,19.999876,619.919187,25.037071


In [58]:
df

,cab_type,source,destination,car_type,date_time,weekday,rush_hour,is_raining,temp_groups,surge_multiplier,price,base_price,estimated_eta,estimated_a,estimated_b,estimated_demand,dynamic_price
0,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 04:43:18.387,1,0,0,40,1.0,9.5,6.5,0.224242,0.057036,8.622001,8.656428,19.117349
1,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 04:49:39.601,1,0,0,40,1.0,9.5,6.5,0.224242,0.057036,8.622001,8.656428,19.117349
2,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 06:25:02.313,1,0,0,40,1.0,8.5,6.5,0.224242,0.057036,8.622001,8.657298,19.118616
3,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 06:30:02.379,1,0,0,40,1.0,7.0,6.5,0.224242,0.057036,8.622001,8.658868,19.120905
4,Uber,South Station,Theatre District,Wheel Chair Accessible,2018-11-26 07:16:52.978,1,1,0,40,1.0,7.0,6.5,0.224242,0.057036,8.622001,8.658868,19.120905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574173,Uber,Northeastern University,Back Bay,Luxury SUV,2018-12-17 04:30:06.790,1,0,0,30,1.0,25.0,25.0,0.000002,0.999873,19.999876,20.999741,25.001256
574174,Lyft,Northeastern University,Back Bay,Luxury SUV,2018-12-17 05:00:04.215,1,0,0,30,1.0,26.0,25.0,0.000002,0.999873,19.999876,20.999741,25.001256
574175,Uber,Northeastern University,Back Bay,Luxury SUV,2018-12-17 05:20:10.955,1,0,0,30,1.0,26.5,25.0,0.000002,0.999873,19.999876,20.999741,25.001256
574176,Uber,Northeastern University,Back Bay,Luxury SUV,2018-12-17 05:50:10.584,1,0,0,30,1.0,26.5,25.0,0.000002,0.999873,19.999876,20.999741,25.001256
